In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/SINDy_tutorial')
path = '/content/drive/MyDrive/SINDy_tutorial/'

In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
os.chdir(path)
from SINDy_tools import calc_gradients, sequentially_thresholded_LSQ
from SINDy_tools import get_vector_test_functions, integrate_3d

In [4]:
f = h5py.File(path+'test.h5', 'r')
dx = f['dx'][()]
dy = dx
dt = f['dt'][()]
Qf = np.moveaxis(f['qtensor'][::4], 0, -1)
Qf = np.array([[Qf[0],Qf[1]],[Qf[1],-Qf[0]]])
uf = np.moveaxis(f['velocity'][::4], 0, -1)
Pf = np.moveaxis(f['pressure'][::4], 0, -1)
f.close()

In [5]:
Q = np.copy(Qf)# + np.random.normal(scale=1e-5, loc=0.0, size=Qf.shape)
Q[0,0] = -Q[1,1]
Q[0,1] = Q[1,0]
u = np.copy(uf)
P = np.copy(Pf)

In [13]:
'''All data is arranged as [...,x,y,t], where "..." corresponds to vector terms'''
lx, ly, lt = Q.shape[-3:]

numSamples = 1000
numDataPoints = lx *ly *lt
sample = np.arange(numDataPoints)
np.random.shuffle(sample)
sample = sample[:numSamples]

wx = 20
wy = 20
wt = 15

vec_w, lap_w, grad_w, dt_w = get_vector_test_functions(wx, wy, wt, dx, dy, dt)

In [14]:
lhs_term = '∇²u'
rhs_term_names = []
rhs_term_names.append('∂ₜu')
rhs_term_names.append('u')
rhs_term_names.append('∇.Q')
rhs_term_names.append('Q.u')
rhs_term_names.append('(u.u)u')
rhs_term_names.append('∇(Q:Q)')
#rhs_term_names.append('∇P')
# term_names.append('(∇.u)u')

rhs_term_names = np.array(rhs_term_names)
numTerms = rhs_term_names.shape[0]

In [19]:
lap_u = calc_gradients(u, lap_order=[1], dh=(dx,dx))
grad_Q = calc_gradients(Q, grad_order=[1], dh=(dx,dx))
grad_P = calc_gradients(P, grad_order=[1], dh=(dx,dx))
dt_u = np.gradient(u, dt, axis=-1)

lhs = np.zeros(numSamples, dtype=float)
rhs = np.zeros((numTerms, numSamples), dtype=float)

for sample in range(numSamples):
    xi = np.random.randint(wx, lx-wx)
    yi = np.random.randint(wy, ly-wy)
    ti = np.random.randint(wt, lt-wt)
    s = (..., slice(xi-wx, xi+wx+1), slice(yi-wy, yi+wy+1), slice(ti-wt, ti+wt+1))

    # LHS
    tmp = np.einsum('a...,a...', lap_w, u[s])
    tmp = integrate_3d(tmp, dx, dy, dt)
    lhs[sample] = tmp

    # RHS
    term = 0
    '''term_names.append('d_t u')'''
    tmp = np.einsum('a...,a...', dt_w, u[s])
    tmp = integrate_3d(tmp, dx, dy, dt)
    rhs[term,sample] = tmp#[0]+tmp[1]
    term += 1
    '''term_names.append('u')'''
    tmp = np.einsum('a...,a...', vec_w, u[s])
    tmp = integrate_3d(tmp, dx, dy, dt)
    rhs[term,sample] = tmp
    term += 1
    '''term_names.append('∇.Q')'''
    tmp = np.einsum('ab...,ba...->...',grad_w, Q[s])
    tmp = integrate_3d(tmp, dx, dy, dt)
    rhs[term,sample] = tmp
    term += 1
    '''term_names.append('Q.u')'''
    tmp = np.einsum('ba...,a...,b...', Q[s], u[s], vec_w)
    tmp = integrate_3d(tmp)
    rhs[term,sample] = tmp
    term += 1
    '''term_names.append('(u.u)u')'''
    tmp = u[s]
    tmp = np.einsum('a...,a...,b...,b...', tmp, tmp, tmp, vec_w)
    tmp = integrate_3d(tmp)
    rhs[term,sample] = tmp
    term += 1
    '''term_names.append('∇(Q:Q)')'''
    tmp = 2 *np.einsum('cab...,ba...,c...', grad_Q[s], Q[s], vec_w)
    tmp = integrate_3d(tmp)
    rhs[term,sample] = tmp
    term += 1
rhs = rhs.T

In [ ]:
coeff, R2 = sequentially_thresholded_LSQ(rhs, lhs)

plt.plot(R2, '-o')

In [ ]:
I = coeff != 0
fitTerms = 2
print('R^2=%0.8f'%R2[fitTerms-1])
print(coeff[fitTerms-1][I[fitTerms-1]])
print(rhs_term_names[I[fitTerms-1]])